# Corrección extremal por Weather Types para la Precipitación en Nueva Zelanda

En este notebook, se aplica la corrección extremal tomando como extremos los máximos anuales.

Librerias:

In [1]:
import numpy as np
import pandas as pd
import os
import sys
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import xarray as xr
import numdifftools as ndt
from scipy import stats 

os.getcwd()

'c:\\Users\\user\\Victor\\Trabajo\\ExtremeCorrection\\Notebooks\\Examples'

In [2]:
variable = "Prec"
os.chdir("../../")
print(os.getcwd())

c:\Users\user\Victor\Trabajo\ExtremeCorrection


## Tratamiento de datos

Carga de datos históricos de la Precipitación

In [3]:
data = xr.open_dataset("Data/Precipitation_NZ/ds_hist_andrea_WT.nc")

In [4]:
data

<xarray.Dataset>
Dimensions:  (time: 675)
Coordinates:
  * time     (time) datetime64[ns] 2002-06-05T09:00:00 ... 2023-05-20T20:00:00
Data variables:
    A        (time) float64 ...
    dwt      (time) int64 ...

In [5]:
df_hist = pd.DataFrame({
    "Prec": data.A.values,
    "wt": data.dwt.values,
    "time": data.time
})

In [6]:
df_hist['yyyy'] = df_hist['time'].dt.year
df_hist['mm'] = df_hist['time'].dt.month
df_hist['dd'] = df_hist['time'].dt.day
df_hist['hh'] = df_hist['time'].dt.hour

In [7]:
df_hist.head()

,Prec,wt,time,yyyy,mm,dd,hh
0,3.993329,31,2002-06-05 09:00:00,2002,6,5,9
1,3.160583,38,2002-06-10 22:00:00,2002,6,10,22
2,11.851844,35,2002-06-17 08:00:00,2002,6,17,8
3,17.241834,46,2002-06-29 15:00:00,2002,6,29,15
4,17.427513,48,2002-07-01 04:00:00,2002,7,1,4


## Carga de la serie simulada

In [8]:
prec_nz_sim = xr.open_dataset("Data\\Precipitation_NZ\\ds_sims_andrea_5000years_WT.nc")
nsim = prec_nz_sim.sim.values

In [9]:
prec_nz_sim

<xarray.Dataset>
Dimensions:  (sim: 10, index: 1826205)
Coordinates:
  * index    (index) int64 0 1 2 3 4 ... 1826191 1826192 1826193 1826194 1826195
  * sim      (sim) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    A        (sim, index) float64 ...
    dwt      (sim, index) int64 ...

In [10]:
np.round(prec_nz_sim.index.values/365.241).astype(int)

array([   0,    0,    0, ..., 5000, 5000, 5000])

In [11]:
dict_df_sim = {}
for sim in nsim:
    dict_df_sim[sim] = pd.DataFrame({
        variable: np.nan_to_num(prec_nz_sim.A.values[sim,:]),
        "time": prec_nz_sim.index.values/365.241,
        "yyyy": np.round(prec_nz_sim.index.values/365.241).astype(int)
    })

In [12]:
dict_df_sim[0]

,Prec,time,yyyy
0,13.661712,0.000000,0
1,18.648568,0.002738,0
2,0.000000,0.005476,0
3,71.097869,0.008214,0
4,0.000000,0.010952,0
...,...,...,...
1826200,0.000000,4999.961669,5000
1826201,0.000000,4999.964407,5000
1826202,0.000000,4999.967145,5000
1826203,41.762109,4999.969883,5000


## Extreme Correction

In [13]:
from src.extremecorrection import ExtremeCorrection

config = {
    'var': variable,
    'time_var': 'time',
    'yyyy_var': 'yyyy',
    'mm_var': 'mm',
    'dd_var': 'dd',
    'freq': 365.25,
    'folder': 'Figures/PrecNZ'
}

pot_config = {
    'n0': 10,
    'min_peak_distance': 2,
    'init_threshold': 0.1,
    'siglevel': 0.01,
    'plot_flag': True
}

In [14]:
extr_corr = ExtremeCorrection(
    data_hist = df_hist,
    data_sim = dict_df_sim[0],
    config = config,
    pot_config = pot_config,
    conf_level = 0.95,
    tolerance = 0.8
)

Applying correction to AnnMax. Poisson parameters difference: 4.043133333333333


### Apply historical correction

In [15]:
extr_corr.apply_correction(fit_diag=True)   # fit_diag = True, to plot the diagnostics plots (QQ-Plot and PP-plot)
extr_corr.return_period_plot()              # Default to plot only the no corrected data 

In [16]:
extr_corr.parameters    # If GEV method, shape parameter has changed the sign

[69.43539510624967, 25.00926904097569, -0.1951493453595903]

In [17]:
print(f"Historical Poisson Parameter: {extr_corr.poiss_parameter}")
print(f"Simulated Poisson Parameter: {extr_corr.sim_poiss_parameter}")

Historical Poisson Parameter: 4.666666666666667
Simulated Poisson Parameter: 8.7098


### Apply simulated correction

In [18]:
extr_corr.apply_sim_correction()   # Apply the correction in the simulated data
extr_corr.sim_return_period_plot()                        # Default to plot the corrected and no corrected data
extr_corr.interval_return_period_plot(alpha=0.2)               # Comparative of no corrected vs corrected simulated data in the historical range
# extr_corr.time_series_plot(sim=False)                     # Time series of both corrected and no-corrected data

## Loop over 10 simulations

In [19]:
for sim in nsim:
    print(f"Simulation {sim}")

    # Configuration
    config = {
        'var': variable,
        'time_var': 'time',
        'yyyy_var': 'yyyy',
        'mm_var': 'mm',
        'dd_var': 'dd',
        'freq': 365.25,
        'folder': f'Figures/PrecNZ_all/Sim{sim}'
    }

    pot_config = {
        'n0': 10,
        'min_peak_distance': 2,
        'init_threshold': 0.1,
        'siglevel': 0.01,
        'plot_flag': True
    }

    # Define extreme correction class
    extr_corr = ExtremeCorrection(
        data_hist = df_hist,
        data_sim = dict_df_sim[sim],
        config = config,
        pot_config = pot_config,
        conf_level = 0.95,
        tolerance = 0.8
    )

    extr_corr.apply_correction(fit_diag=True)   # fit_diag = True, to plot the diagnostics plots (QQ-Plot and PP-plot)
    extr_corr.return_period_plot()              # Default to plot only the no corrected data 

    
    print(f"Method: {extr_corr.method}, parameters {extr_corr.parameters}")    # If GEV method, shape parameter has changed the sign


    extr_corr.apply_sim_correction()   # Apply the correction in the simulated data
    extr_corr.sim_return_period_plot()                        # Default to plot the corrected and no corrected data
    extr_corr.interval_return_period_plot(alpha=0.2)               # Comparative of no corrected vs corrected simulated data in the historical range
    # extr_corr.time_series_plot(sim=False)                     # Time series of both corrected and no-corrected data
    
    print("-----------------------\n")

Simulation 0
Applying correction to AnnMax. Poisson parameters difference: 4.043133333333333
Method: AnnMax, parameters [69.43539510624967, 25.00926904097569, -0.1951493453595903]
-----------------------

Simulation 1
Applying correction to AnnMax. Poisson parameters difference: 4.023533333333334
Method: AnnMax, parameters [69.43539510624967, 25.00926904097569, -0.1951493453595903]
-----------------------

Simulation 2
Applying correction to AnnMax. Poisson parameters difference: 4.125133333333333
Method: AnnMax, parameters [69.43539510624967, 25.00926904097569, -0.1951493453595903]
-----------------------

Simulation 3
Applying correction to AnnMax. Poisson parameters difference: 4.0511333333333335
Method: AnnMax, parameters [69.43539510624967, 25.00926904097569, -0.1951493453595903]
-----------------------

Simulation 4
Applying correction to AnnMax. Poisson parameters difference: 4.033733333333333
Method: AnnMax, parameters [69.43539510624967, 25.00926904097569, -0.1951493453595903]